In [1]:
!pip install transformers

## Sentiment Analysis - Using DistillBert
- A Default DistilBERT model - sshleifer/distilbart-cnn-12-6 (which is a distilled version of BERT — both are LLMs)
- Fine-tuned for sentiment classification
- Runs via the Hugging Face transformers pipeline (which wraps the model and tokenizer)

In [19]:
import pandas as pd
from transformers import pipeline
analyzer = pipeline("sentiment-analysis")
# default model is : sshleifer/distilbart-cnn-12-6
print("\n\n#########################################")
print("Defaul model :", analyzer.model.name_or_path)
print("#########################################\n\n")
data = pd.DataFrame({"feedback": ["Love this!", "Hate the new design"]})
data['sentiment'] = data['feedback'].apply(analyzer)
data

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu




#########################################
Defaul model : distilbert/distilbert-base-uncased-finetuned-sst-2-english
#########################################




,feedback,sentiment
0,Love this!,"[{'label': 'POSITIVE', 'score': 0.999874591827..."
1,Hate the new design,"[{'label': 'NEGATIVE', 'score': 0.999741017818..."


## Text Summarizer using HuggingFace Gated Models
- current URL https://huggingface.co/models
- some models are gated, i.e. they need a special request access to use the models
- we are using `Mistral-7B-Instruct-v0.3` for doing summarization of famous story *Kabulliwalah* by *RabindraNath Tagor*
- hf-token is needed to use the model the step by step instructions as as given below

---

### **Create Account / Login**

* Go to [https://huggingface.co](https://huggingface.co)
* Click **Sign Up** (or **Log In** if you already have an account)
* Complete registration or login with GitHub/Google or email/password

---

### **Get HF Token**

* Once logged in, click on your profile icon (top-right corner)
* From the dropdown, click **Settings**
* In the sidebar, click **Access Tokens**
* Click the **New Token** button
* Provide a **name** (e.g., `my-token`) and choose a **role** (for most uses, "Read" is sufficient)
* Click **Generate Token**
* Copy the token shown — it will start with `hf_...`

---

### **Use the Token (e.g., in CLI or Code)**

To log in via terminal/CLI:

```bash
huggingface-cli login
```

Then paste the token when prompted.

To use it in Python:

```python
from huggingface_hub import login
login("hf_YourTokenHere")
```



In [12]:
import requests

API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"

HF_TOKEN =  open("/content/hf_token.txt", "r").read()
headers = {
    "Authorization": f"Bearer {HF_TOKEN}"
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

prompt = (
    "### Instruction:\n"
    "Summarize the following story in 3-4 sentences.\n\n"
    "### Input:\n"
)

# Read and truncate your input
with open("/content/radindranath_tagor_cabuliwala.txt", "r", encoding="utf-8") as f:
    story_text = f.read()[:1500]

full_prompt = prompt + story_text + "\n\n### Response:\n"

output = query({
    "inputs": full_prompt,
    "parameters": {
        "max_new_tokens": 150,
        "do_sample": False,
        "return_full_text": False
    }
})

# Print the response
print("Summary:\n", output[0]['generated_text'])


Summary:
  The story is about a father and his five-year-old daughter, Mini, who is chatty and inquisitive. Despite her mother's disapproval, the father encourages her talk. One morning, Mini interrupts the father while he is writing, questioning the pronunciation of a word, the existence of an elephant in the clouds, and her relationship with her mother. The father humorously answers her questions and sends her to play.


## Google Gemini API
---
Typiclally we can use `gemini-2.0-flash` or `gemini-1.5-flash`
---

### STEP 1: Get a Gemini API Key (Free)

* Go to: [https://aistudio.google.com/app/apikey](https://aistudio.google.com/app/apikey)
* Sign in with your Google account
* Click **“Create API key”**
* Copy the key — it will start with `AIza...` or `AIzaSy...`

---

### STEP 2: Install Required Python Package

```bash
pip install google-generativeai
```

---

### Free Usage Limits (as of June 2025):

| Model            | RPM (Requests/Min) | RPD (Requests/Day) |
| ---------------- | ------------------ | ------------------ |
| gemini-1.5-flash | 15                 | 1,500              |
| gemini-1.5-pro   | 2                  | 50                 |

---
### Google Gemini AI API documents
- [General Documents](https://ai.google.dev/gemini-api/docs)
- **[API Rate Limits](https://ai.google.dev/gemini-api/docs/rate-limits)**

## Use Google Gemini AI API using gemini2.0

In [20]:
! pip install google-generativeai

In [22]:

GOOGLE_API_KEY =  open("/content/google_gemini_key.txt", "r").read()

import google.generativeai as genai

# Set up the API key
genai.configure(api_key=GOOGLE_API_KEY)

# Load the model (Gemini 1.5 Flash is free and faster)
model = genai.GenerativeModel("gemini-2.0-flash")

# Send a prompt
response = model.generate_content("What are the benefits of reinforcement learning?")

print(response.text)


Reinforcement Learning (RL) offers several compelling benefits, making it a powerful tool for a wide range of applications. Here's a breakdown of the key advantages:

**1. Learning Optimal Strategies Automatically:**

*   **Autonomous Policy Creation:** RL algorithms learn policies (strategies) to achieve a goal without explicit programming. The agent explores the environment and learns from its interactions, gradually improving its performance. This is particularly useful in complex scenarios where defining optimal rules manually is difficult or impossible.
*   **Adaptability to Change:** RL agents can adapt to changes in the environment or task. If the environment shifts or the goal evolves, the agent can continue to learn and optimize its policy, ensuring continued performance.

**2. Dealing with Complex and Dynamic Environments:**

*   **Handles Uncertainty:** RL is well-suited for environments with uncertainty or stochasticity (randomness). It can learn robust policies that perfor

## Multiturn Chat Context using Gemini2.0

In [23]:
chat = model.start_chat(history=[])
chat.send_message("What's the capital of Germany?")
chat.send_message("And its population?")
print(chat.history[-1].parts[0].text)


The population of Berlin is estimated to be around **3.7 million** as of 2023. Keep in mind that population figures are always estimates and can change.



In [24]:
chat.send_message("And what are its states?")
print(chat.history[-1].parts[0].text)

Berlin is a city-state, which means it is both a city and one of the 16 states of Germany. So, in this case, **Berlin *is* the state.**

Germany is a federal republic made up of 16 states (Bundesländer). These states have their own constitutions, parliaments, and governments, but are also subject to the federal government.



### Automate a Process using LLM (Google Gemini AI API)

In [ ]:
! pip install langchain google-generativeai langchain-google-genai beautifulsoup4 requests langchain-community



In [37]:
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import WebBaseLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

# Your API key explicitly defined here
API_KEY =  open("/content/google_gemini_key.txt", "r").read()

# Scrap the transcripts
url = "https://lexfridman.com/sundar-pichai-transcript#chapter1_introduction"
loader = WebBaseLoader(url)
docs = loader.load()


transcript_text = "\n".join(doc.page_content for doc in docs)

with open("sundar_pichai_transcript.txt", "w", encoding="utf-8") as f:
    f.write(transcript_text)

# Trim transcript text if necessary for token limit
transcript_input = transcript_text[:10000]

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
    google_api_key=API_KEY
)

prompt = ChatPromptTemplate.from_template("""
Summarize the following transcript into exactly 100 words, focusing on Sundar Pichai's key insights, leadership views, and thoughts on technology:

Transcript:
{transcript}
""")

chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.invoke({"transcript": transcript_input})

# Display the summary
print("Summary (100 words):\n")
print("\n".join(summary["text"].split(".")))

Summary (100 words):

Sundar Pichai's journey from a humble upbringing in India, lacking basic amenities, profoundly shaped his perspective on technology's transformative power
  He emphasizes the importance of listening to one's heart when choosing a career path, highlighting the ease of success when passion is involved
  Pichai expresses optimism about humanity's ability to address the risks associated with advanced AI, believing collective effort can mitigate potential "doom" scenarios
 His leadership philosophy centers on recognizing technology's life-changing impact, a belief rooted in his personal experiences and driving his vision for Google and Alphabet
  He views technological progress as exponential and often overlooked, urging appreciation for its advancements

